Leemos el archivo recibido

In [1]:
import pandas as pd
dftrain = pd.read_parquet('../proyecto_ML/train.parquet')

Se cuentan las propiedades duplicadas en base a sus caracteristicas, se excluye el precio ya que este puede ser modificado por mejoras, inflación u otros factores.

In [2]:
dupes = dftrain.duplicated(subset=['image_url','sqfeet','beds','baths','wheelchair_access','parking_options','laundry_options','state','region'], keep='first')
dupes.sum()

125793

Se obtienen duplicados con diferentes condiciones de ordenamiento para verificar los datos de descripción.

In [3]:
#Obtener duplicados
dfIds = dftrain['id']
dfdupes = dftrain[dfIds.isin(dfIds[dftrain.duplicated(subset=['image_url','sqfeet','beds','baths','wheelchair_access','parking_options','laundry_options','state','region'])])].sort_values('description')
dfdupes[['id', 'price','description', 'image_url']].sort_values('image_url').head(100)

,id,price,description,image_url
262100,7028985169,960,Welcome home to Fort Lane! # of Bedrooms: 2 ...,https://images.craigslist.org/00000_11wT9EUycb...
334182,7035784866,791,Autumn Ridge Apartments II â Starting at $79...,https://images.craigslist.org/00000_16h4w7k2zi...
310552,7043510251,950,Generously sized one bedroom one bath apartmen...,https://images.craigslist.org/00000_1AJZPS1aUT...
313041,7027637333,631,Villa West is accepting applications for our w...,https://images.craigslist.org/00000_1AuetpjLvW...
137219,7051102343,631,Villa West is accepting applications for our w...,https://images.craigslist.org/00000_1AuetpjLvW...
...,...,...,...,...
71095,7031402971,800,Lincoln Heights Apartments KEY FEATURES Bedroo...,https://images.craigslist.org/00000_2XyOos053d...
256272,7039699134,750,Lincoln Heights Apartments KEY FEATURES Bedroo...,https://images.craigslist.org/00000_2XyOos053d...
70798,7050720117,1900,***** THIS HOUSE IS AVAILABLE JULY 2020 ***** ...,https://images.craigslist.org/00000_2bomz1oXuc...
226786,7050719956,1900,***** THIS HOUSE IS AVAILABLE JULY 2020 *****...,https://images.craigslist.org/00000_2bomz1oXuc...


Se extraen los duplicados manteniendo el valor mas alto de cada grupo duplicado y se verifican las eliminaciones por ID.

In [4]:
#Sacar duplicados manteniendo el valor mas alto y verificar cual queda
dftrain2 = dftrain.sort_values(by=['price'], ascending=False,ignore_index=False)
dftrain2 = dftrain2.drop_duplicates(subset=['image_url','sqfeet','beds','baths','wheelchair_access','parking_options','laundry_options','state','region'], keep='first')
dftrain2.loc[dftrain2['id'].isin([7031402971,7039699134])]


,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
71095,7031402971,https://wichitafalls.craigslist.org/apa/d/wich...,wichita falls,https://wichitafalls.craigslist.org,800,apartment,725,2,1.5,1,...,0,0,0,laundry on site,off-street parking,https://images.craigslist.org/00000_2XyOos053d...,Lincoln Heights Apartments KEY FEATURES Bedroo...,33.9519,-98.5145,tx


Corroborar cantidad de eliminados.

In [5]:
#Corroborar original, duplicados y restantes
print(len(dftrain.index))
print(len(dftrain2.index))
print(len(dfdupes.index))

346479
220686
125793


Verificar cantidad de nulos restantes.

In [6]:
dftrain2.isna().sum()

id                             0
url                            0
region                         0
region_url                     0
price                          0
type                           0
sqfeet                         0
beds                           0
baths                          0
cats_allowed                   0
dogs_allowed                   0
smoking_allowed                0
wheelchair_access              0
electric_vehicle_charge        0
comes_furnished                0
laundry_options            47515
parking_options            79270
image_url                      0
description                    2
lat                         1088
long                        1088
state                          0
dtype: int64

Se eliminan las filas con coordenadas fuera de Estados Unidos como se hizo en el EDA.
*En este caso no se eliminan los nulos*

In [7]:
df2 = dftrain2[ ((dftrain['lat'] < 49.38) & (dftrain2['lat'] > 24.52) & (dftrain2['long'] < -66.94 ) & (dftrain2['long'] > -124.73 )) | (dftrain2['state'] == 'hi') | (dftrain2['state'] == 'ak') ]

print('Cantidad original: ' + str(len(dftrain2.index)))
print('Cantidad filtrada: ' + str(len(df2.index)))

Cantidad original: 220686
Cantidad filtrada: 219509


C:\Users\Nahuel\AppData\Local\Temp\ipykernel_24176\1714816055.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = dftrain2[ ((dftrain['lat'] < 49.38) & (dftrain2['lat'] > 24.52) & (dftrain2['long'] < -66.94 ) & (dftrain2['long'] > -124.73 )) | (dftrain2['state'] == 'hi') | (dftrain2['state'] == 'ak') ]


Se eliminan los outliers como se hizo en el EDA.

In [8]:
df_filtered = df2.loc[(df2['price'].between(50,3000)) & (df2['sqfeet'].between(100,2500)) & (df2['beds'].between(0,100)) & (df2['baths'].between(0,10))]
print("cantidad de precios excluidos")
print(219486 - len(df2[df2['price'].between(50,3000)].index))
print("cantidad de pies cuadrados excluidos")
print(219486 - len(df2[df2['sqfeet'].between(100,2500)].index))
print("cantidad de camas excluidos")
print(219486 - len(df2[df2['beds'].between(0,100)].index))
print("cantidad de banos excluidos")
print(219486 - len(df2[df2['baths'].between(0,10)].index))
print("Cantidad de filas final :")
print(len(df_filtered.index))

cantidad de precios excluidos
5666
cantidad de pies cuadrados excluidos
3081
cantidad de camas excluidos
-20
cantidad de banos excluidos
-20
Cantidad de filas final :
211364


Se verifican los valores de dos columnas.

In [9]:
print(pd.unique(df_filtered['laundry_options'].sort_values()))
print(pd.unique(df_filtered['parking_options'].sort_values()))

['laundry in bldg' 'laundry on site' 'no laundry on site' 'w/d hookups'
 'w/d in unit' None]
['attached garage' 'carport' 'detached garage' 'no parking'
 'off-street parking' 'street parking' 'valet parking' None]


Se ajustan los valores nulos para que coincidan.

In [10]:
import numpy as np
df_filtered.laundry_options.replace({np.nan: None}, inplace=True)
df_filtered.parking_options.replace({np.nan: None}, inplace=True)

C:\Users\Nahuel\AppData\Local\Temp\ipykernel_24176\12136549.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.laundry_options.replace({np.nan: None}, inplace=True)
C:\Users\Nahuel\AppData\Local\Temp\ipykernel_24176\12136549.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.parking_options.replace({np.nan: None}, inplace=True)


Se quitan las filas con regiones que no estan incluidas en el archivo de pruebas.

In [11]:
print(len(df_filtered.index))
df_filtered = df_filtered[~df_filtered['region'].isin(['fort smith, AR','kansas city','salina','southwest TX','st louis','western IL'])]
print(len(df_filtered.index))

211364
211282


Se genera el archivo csv para ser procesado.

In [13]:
df_filtered.to_csv('../Datasets/deSegundo.csv', index=False)